<a href="https://colab.research.google.com/github/d9249/DACON/blob/main/CVLC_06_VGG16(public_%2C_private_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/test.csv')

In [3]:
!mkdir images_train
!mkdir images_train/0
!mkdir images_train/1
!mkdir images_train/2
!mkdir images_train/3
!mkdir images_train/4
!mkdir images_train/5
!mkdir images_train/6
!mkdir images_train/7
!mkdir images_train/8
!mkdir images_train/9
!mkdir images_test

In [4]:
import cv2

for idx in range(len(train)) :
    img = train.loc[idx, '0':].values.reshape(28, 28).astype(int)
    digit = train.loc[idx, 'digit']
    cv2.imwrite(f'./images_train/{digit}/{train["id"][idx]}.png', img)

In [5]:
import cv2

for idx in range(len(test)) :
    img = test.loc[idx, '0':].values.reshape(28, 28).astype(int)
    cv2.imwrite(f'./images_test/{test["id"][idx]}.png', img)

In [6]:
import tensorflow as tf
VGG16_model =  tf.keras.applications.vgg16.VGG16(weights=None, include_top=True, input_shape=(224, 224, 1), classes=10)

In [7]:
# from tensorflow.keras.optimizers import Adam
VGG16_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1)

train_generator = datagen.flow_from_directory('./images_train', target_size=(224,224), color_mode='grayscale', class_mode='categorical', subset='training')
val_generator = datagen.flow_from_directory('./images_train', target_size=(224,224), color_mode='grayscale', class_mode='categorical', subset='validation')

Found 1642 images belonging to 10 classes.
Found 406 images belonging to 10 classes.


In [9]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_VGG16.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [11]:
VGG16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
VGG16_model.fit_generator(train_generator, epochs=500, validation_data=val_generator, callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
52/52 [==============================] - 37s 325ms/step - loss: 2.3050 - accuracy: 0.1005 - val_loss: 2.3007 - val_accuracy: 0.1133

Epoch 00001: val_accuracy improved from -inf to 0.11330, saving model to /content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_VGG16.h5
Epoch 2/500
52/52 [==============================] - 13s 254ms/step - loss: 2.3017 - accuracy: 0.1139 - val_loss: 2.3001 - val_accuracy: 0.1133

Epoch 00002: val_accuracy did not improve from 0.11330
Epoch 3/500
52/52 [==============================] - 13s 254ms/step - loss: 2.3017 - accuracy: 0.1139 - val_loss: 2.3000 - val_accuracy: 0.1133

Epoch 00003: val_accuracy did not improve from 0.11330
Epoch 4/500
52/52 [==============================] - 13s 248ms/step - loss: 2.3012 - accuracy: 0.1139 - val_loss: 2.3001 - val_accuracy: 0.1133

Epoch 00004: val_accuracy did not improve from 0.11330
Epoch 5/500
52/52 [==============================] - 13s 248ms/step - loss: 2.3018 - accuracy: 0.1139 - val_loss: 2.3001

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(VGG16_model.history.history["accuracy"], label='VGG16_acc')
plt.plot(VGG16_model.history.history["val_accuracy"], label='VGG16_val')

plt.legend()
plt.show()

In [ ]:
InceptionV3_model = tf.keras.models.load_model('/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_VGG16.h5', compile=False)

In [ ]:
!mkdir images_test/none
!mv images_test/*.png images_test/none

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory('./images_test', target_size=(224,224), color_mode='grayscale', class_mode='categorical', shuffle=False)

In [ ]:
VGG16_predict = VGG16_model.predict_generator(test_generator).argmax(axis=1)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/submission.csv')
submission.head()

In [ ]:
import numpy as np
mylist = []

for i in range(len(submission)):
    name =  test_generator.filenames
    id = name[i].split('/')[1].rstrip('.').split('.')[0]
    mylist.append(id)
print(mylist)

In [ ]:
for i in range(len(submission)):
    submission["id"][i] = mylist[i]

In [ ]:
submission["VGG16_predict"] = VGG16_predict
submission.head()

In [ ]:
from collections import Counter

for i in range(len(submission)) :
    predicts = submission.loc[i, ['VGG16_predict']]
    submission.at[i, "digit"] = Counter(predicts).most_common(n=1)[0][0]

submission.head()

In [ ]:
submission = submission[['id', 'digit']]
submission.head()

In [ ]:
from google.colab import files

submission.to_csv('/content/drive/MyDrive/DACON_CVLC/Submission/VGG16_model.csv', index=False)
files.download('/content/drive/MyDrive/DACON_CVLC/Submission/VGG16_model.csv')